In [1]:
"""
A script to validate query fixes (i.e. do the fixes actually result in stable queries).
"""
import os
import sys

original_cwd = os.getcwd()
print(original_cwd)

sys.path.append("/home/yizhou7/mariposa/src")
os.chdir("/home/yizhou7/mariposa/")

import numpy as np
from statistics import mean, median
import pandas as pd
from pathlib import Path
import glob
import matplotlib.pyplot as plt

from debugger.proof_analyzer import ProofAnalyzer
from debugger.mutant_info import TraceFailure
from evaluator import Evaluator, BenchViewer

from base.solver import RCode
from debugger.debugger import Debugger3
from enum import Enum

from utils.analysis_utils import Categorizer

# set this to true if I need to move folders over and set up a proj
move_folders = False

/home/amarshah/mariposa/src


In [13]:
revs = BenchViewer({'data/projs/anvil/base.z3/fluent-smt-fluent_controller__fluentbit__proof__liveness__resource_match__lemma_from_after_get_resource_step_to_after_update_resource_step.smt2', 'data/projs/anvil/base.z3/fluent-smt-fluent_controller__fluentbit__proof__helper_invariants__validation.3.smt2', 'data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-multilog__setup_v.5.smt2', 'data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-multilog__inv_v.11.smt2', 'data/projs/vsystemsnew/base.z3/mimalloc-smt-page_organization__PageOrg.69.smt2', 'data/projs/verismo.dedup/base.z3/snp__ghcb__proto_page.141.smt2', 'data/projs/anvil/base.z3/zookeeper-smt-zookeeper_controller__proof__liveness__stateful_set_match.2.smt2', 'data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-log__inv_v.13.smt2', 'data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-multilog__inv_v.16.smt2', 'data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-multilog__inv_v.15.smt2', 'data/projs/verismo.dedup/base.z3/bsp__ap.3.smt2', 'data/projs/verismo.dedup/base.z3/vbox__vbox.4.smt2', 'data/projs/anvil/base.z3/zookeeper-smt-zookeeper_controller__proof__helper_invariants__proof__lemma_eventually_always_no_delete_resource_request_msg_in_flight.smt2', 'data/projs/anvil/base.z3/zookeeper-smt-rabbitmq-smt-rabbitmq_controller__proof__helper_invariants__validation__lemma_always_stateful_set_in_create_request_msg_satisfies_unchangeable.smt2', 'data/projs/splinterdb/base.z3/allocation_layer__LikesBetree_v.18.smt2', 'data/projs/anvil/base.z3/zookeeper-smt-rabbitmq-smt-rabbitmq_controller__proof__liveness__stateful_set_match__lemma_from_after_get_stateful_set_step_to_after_update_stateful_set_step.smt2', 'data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-log__logimpl_v.6.smt2', 'data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-multilog__inv_v.2.smt2', 'data/projs/splinterdb/base.z3/allocation_layer__LikesBetree_v__LikesBetree.3.smt2', 'data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-log__inv_v.2.smt2', 'data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-multilog__inv_v.9.smt2', 'data/projs/verismo.dedup/base.z3/boot__init__e820_init.4.smt2', 'data/projs/anvil/base.z3/zookeeper-smt-rabbitmq-smt-rabbitmq_controller__proof__helper_invariants__proof__lemma_eventually_always_every_resource_create_request_implies_at_after_create_resource_step.smt2', 'data/projs/anvil/base.z3/fluent-smt-fluent_controller__fluentbit__proof__liveness__daemon_set_match__lemma_from_after_get_daemon_set_step_to_after_update_daemon_set_step.smt2', 'data/projs/anvil/base.z3/zookeeper-smt-zookeeper_controller__proof__helper_invariants__validation.3.smt2', 'data/projs/vsystemsnew/base.z3/mimalloc-smt-segment__segment_span_free_coalesce_before.smt2', 'data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-log__layout_v.30.smt2', 'data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-multilog__inv_v.14.smt2', 'data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-multilog__inv_v.13.smt2', 'data/projs/anvil/base.z3/fluent-smt-fluent_controller__fluentbit_config__proof__liveness__resource_match.5.smt2', 'data/projs/anvil/base.z3/zookeeper-smt-rabbitmq-smt-rabbitmq_controller__proof__liveness__resource_match.1.smt2', 'data/projs/vsystemsnew/base.z3/mimalloc-smt-commit_segment.1.smt2', 'data/projs/verismo.dedup/base.z3/snp__cpuid.17.smt2', 'data/projs/anvil/base.z3/fluent-smt-fluent_controller__fluentbit_config__proof__liveness__resource_match.6.smt2', 'data/projs/verismo.dedup/base.z3/vbox__vbox.5.smt2', 'data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-log__inv_v.17.smt2', 'data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-multilog__inv_v.17.smt2', 'data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-multilog__inv_v.7.smt2', 'data/projs/anvil/base.z3/zookeeper-smt-zookeeper_controller__proof__helper_invariants__proof.2.smt2', 'data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-log__inv_v.10.smt2', 'data/projs/anvil/base.z3/zookeeper-smt-rabbitmq-smt-rabbitmq_controller__proof__liveness__stateful_set_match__lemma_stateful_set_state_matches_at_after_update_stateful_set_step.smt2', 'data/projs/vsystemsnew/base.z3/page-table-smt-impl_u__l2_refinement.3.smt2', 'data/projs/vsystemsnew/base.z3/mimalloc-smt-config.35.smt2', 'data/projs/verismo.dedup/base.z3/security__monitor.21.smt2', 'data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-log__inv_v.4.smt2', 'data/projs/verismo.dedup/base.z3/boot__init__e820_init_alloc.1.smt2', 'data/projs/verismo.dedup/base.z3/boot__init__e820_init.2.smt2', 'data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-log__inv_v.12.smt2', 'data/projs/vsystemsnew/base.z3/mimalloc-smt-segment__segment_os_alloc.smt2', 'data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-log__inv_v.5.smt2', 'data/projs/anvil/base.z3/fluent-smt-fluent_controller__fluentbit__proof__helper_invariants__proof.9.smt2', 'data/projs/verismo.dedup/base.z3/pgtable_e__pte.20.smt2', 'data/projs/anvil/base.z3/fluent-smt-fluent_controller__fluentbit_config__proof__liveness__resource_match__lemma_from_after_get_resource_step_to_after_create_resource_step.smt2', 'data/projs/anvil/base.z3/zookeeper-smt-zookeeper_controller__proof__helper_invariants__zookeeper_api__lemma_zk_request_implies_step_helper.smt2', 'data/projs/anvil/base.z3/zookeeper-smt-zookeeper_controller__proof__helper_invariants__proof__lemma_always_no_update_status_request_msg_in_flight_of_except_stateful_set.smt2', 'data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-multilog__start_v.2.smt2', 'data/projs/verismo.dedup/base.z3/pgtable_e__pte.18.smt2', 'data/projs/verismo.dedup/base.z3/allocator__linkedlist.10.smt2', 'data/projs/atmosphere/base.z3/kernel__create_and_share_pages.5.smt2', 'data/projs/anvil/base.z3/zookeeper-smt-rabbitmq-smt-rabbitmq_controller__proof__helper_invariants__proof__lemma_eventually_always_object_in_response_at_after_create_resource_step_is_same_as_etcd.smt2', 'data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-multilog__setup_v.6.smt2', 'data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-log__inv_v.14.smt2', 'data/projs/vsystemsnew/base.z3/mimalloc-smt-segment__span_queue_delete.smt2'})

[WARN] [eval] 66c9bdf69f tested count 1225 < registered 1612 
[WARN] [eval] 82449bd2e8 tested count 112 < registered 338 


In [ ]:
f_modes = Categorizer()

for query in revs.fixable:
    rev: Evaluator = revs[query]
    mi = rev.get_trace_info()
    reason = mi.get_failed_reason()
    f_modes.add_item(reason, query)


In [ ]:
f_modes.finalize()
f_modes.print_status()

| category                  |   count | percentage   |
|---------------------------|---------|--------------|
| TraceFailure.TIMEOUT      |      35 | 55.56 %      |
| TraceFailure.FAST_UNKNOWN |      27 | 42.86 %      |
| TraceFailure.SLOW_UNKNOWN |       1 | 1.59 %       |
| total                     |      63 | 100.00 %     |


In [ ]:
def get_name_hash(filename):
    import hashlib
    return hashlib.sha256(filename.encode()).hexdigest()[0:10]

fast_unknown_files = ["../cache/" + get_name_hash(f) + ".report" for f in f_modes[TraceFailure.FAST_UNKNOWN]]
timeout_files = ["../cache/" + get_name_hash(f) + ".report" for f in f_modes[TraceFailure.TIMEOUT]]
slow_unknown_files = ["../cache/" + get_name_hash(f) + ".report" for f in f_modes[TraceFailure.SLOW_UNKNOWN]]
not_fail_files = ["../cache/" + get_name_hash(f) + ".report" for f in f_modes[TraceFailure.NOT_FAIL]]


In [ ]:
from utils.cache_utils import load_cache


def process_queries(files):
    stabilizing_edits = []
    for file in files:
        print(file)
        report = load_cache(file)
        stabilizing_edits += report.stabilized['edit_path'].to_list()
    return stabilizing_edits


fast_unknown_edits = process_queries(fast_unknown_files)
timeout_edits = process_queries(timeout_files)
slow_unknown_edits = process_queries(slow_unknown_files)
not_fail_edits = process_queries(not_fail_files)
print(slow_unknown_edits)

../cache/89068d3f38.report
../cache/090a2a7d67.report
../cache/025a074d17.report
../cache/b689a8d455.report
../cache/3c5c22d4a1.report
../cache/af029e0bc2.report
../cache/7d8c4302ab.report
../cache/32eaf80bcc.report
../cache/568e167040.report
../cache/e16c5e8c0b.report
../cache/09d24c83cd.report
../cache/f6f3f962c0.report
../cache/92c260e39d.report
../cache/be920877ca.report
../cache/2078a24298.report
../cache/126f0f80f3.report
../cache/5a940edd1b.report
../cache/ed912ce861.report
../cache/492704d0da.report
../cache/815f69b161.report
../cache/9a3bc13b2d.report
../cache/c4ec60f8f9.report
../cache/a896b920ca.report
../cache/2045867a58.report
../cache/e1d2573021.report
../cache/c02ff41a27.report
../cache/fa9020870d.report
../cache/48dbc6d4fa.report
../cache/fa2fdaf6e0.report
../cache/9eaad8947e.report
../cache/2faba7bd1f.report
../cache/a1b4b3fd4a.report
../cache/4d2d4fbc20.report
../cache/fef99080cd.report
../cache/106770d98e.report
../cache/634ab613c3.report
../cache/55b4af82a4.report
.

In [ ]:
import shutil
import os

os.chdir(original_cwd)
print(os.getcwd())

# Define source and target paths
source_dir = "/home/yizhou7/mariposa/"
target_base = "/home/amarshah/mariposa/data/samples"
# target_base = os.getcwd() + "../data/samples/"

# Define mapping of list names to target folders
edit_groups = {
    "fast_unknown": fast_unknown_edits,
    "timeout": timeout_edits,
    "slow_unknown": slow_unknown_edits,
    "not_fail": not_fail_edits,
}

if move_folders:

    # Move files to the respective base.z3/ folders
    for category, file_list in edit_groups.items():
        target_dir = os.path.join(target_base, f"verus_{category}_proj")
        os.makedirs(target_dir, exist_ok=True) 

        for file_name in file_list:
            src = os.path.join(source_dir, file_name)

            # Strip everything before and including the last "/"
            dest_file_name = os.path.basename(file_name)
            dest = os.path.join(target_dir, dest_file_name)

            try:
                if os.path.exists(dest):
                    os.remove(dest)  # Remove existing file before moving
                shutil.copy(src, dest)
                print(f"Moved {src} → {dest}")
            except Exception as e:
                print(f"Error moving {src}: {e}")


/home/yizhou7/mariposa
Moved /home/yizhou7/mariposa/data/projs/singleton_89068d3f38/base.z3/0bbd00e5.smt2 → /home/amarshah/mariposa/data/samples/verus_fast_unknown_proj/0bbd00e5.smt2
Moved /home/yizhou7/mariposa/data/projs/singleton_89068d3f38/base.z3/4f91bee7.smt2 → /home/amarshah/mariposa/data/samples/verus_fast_unknown_proj/4f91bee7.smt2
Moved /home/yizhou7/mariposa/data/projs/singleton_89068d3f38/base.z3/57536093.smt2 → /home/amarshah/mariposa/data/samples/verus_fast_unknown_proj/57536093.smt2
Moved /home/yizhou7/mariposa/data/projs/singleton_89068d3f38/base.z3/799880e0.smt2 → /home/amarshah/mariposa/data/samples/verus_fast_unknown_proj/799880e0.smt2
Moved /home/yizhou7/mariposa/data/projs/singleton_89068d3f38/base.z3/9689b33d.smt2 → /home/amarshah/mariposa/data/samples/verus_fast_unknown_proj/9689b33d.smt2
Moved /home/yizhou7/mariposa/data/projs/singleton_89068d3f38/base.z3/a862c66e.smt2 → /home/amarshah/mariposa/data/samples/verus_fast_unknown_proj/a862c66e.smt2
Moved /home/yizho

In [ ]:
import subprocess


# List of project names
names = ["verus_timeout", "verus_not_fail", "verus_fast_unknown", "verus_slow_unknown"]

if move_folders:
    # Run the commands
    # os.chdir("/home/amarshah/mariposa")
    for name in names:
        print(name)
        command = ["./src/proj_wizard.py", "create", "-i", f"data/samples/{name}_proj", "--new-project-name", name]
        subprocess.run(command, check=True)


verus_timeout
[INFO] output directory is set to data/projs/verus_timeout/base.z3 
[INFO] generated 860 targets in build.ninja 
[1/860] src/smt2action/target/release/mariposa -i data/samples/verus_timeout_proj/b1991cfe.smt2 -o data/projs/verus_timeout/base.z3/b1991cfe.smt2 -a split --convert-comments
query file 0 check-sat command(s) ignored, rest is split into 1 file(s)
[2/860] src/smt2action/target/release/mariposa -i data/samples/verus_timeout_proj/eb9da005.smt2 -o data/projs/verus_timeout/base.z3/eb9da005.smt2 -a split --convert-comments
query file 0 check-sat command(s) ignored, rest is split into 1 file(s)
[3/860] src/smt2action/target/release/mariposa -i data/samples/verus_timeout_proj/9689b33d.smt2 -o data/projs/verus_timeout/base.z3/9689b33d.smt2 -a split --convert-comments
query file 0 check-sat command(s) ignored, rest is split into 1 file(s)
[4/860] src/smt2action/target/release/mariposa -i data/samples/verus_timeout_proj/9d1f38d0.smt2 -o data/projs/verus_timeout/base.z3/9d1

In [2]:
# Running Mariposa on the fast_unknown. Can start here if the project has already been built
import subprocess

os.chdir(original_cwd)
# Move to parent directory
parent_dir = os.path.abspath("..")  # Get absolute path of the parent directory
os.chdir(parent_dir)  # Change working directory

command = ["./src/exper_wizard.py", "multiple", "-i", "data/projs/verus_fast_unknown/base.z3/", "-e", "verus_quick"]
subprocess.run(command, check=True)




[DEBUG] created table verus_fast_unknown_base_z3_z3_4_13_0_exp 
[DEBUG] running ALL of verus_fast_unknown.base.z3 
[DEBUG] adding 45793 tasks 


[INFO] running 253 original queries 
[INFO] running 45793 tasks 


[DEBUG] finished: 2660/45794, elapsed: 0.03 hours, estimated: 02-16 16:30 
[DEBUG] finished: 2661/45794, elapsed: 0.03 hours, estimated: 02-16 16:30 
[DEBUG] finished: 2662/45794, elapsed: 0.03 hours, estimated: 02-16 16:30 
[DEBUG] finished: 2663/45794, elapsed: 0.03 hours, estimated: 02-16 16:30 
[DEBUG] finished: 2666/45794, elapsed: 0.03 hours, estimated: 02-16 16:30 
[DEBUG] finished: 2667/45794, elapsed: 0.03 hours, estimated: 02-16 16:30 
[DEBUG] finished: 2696/45794, elapsed: 0.03 hours, estimated: 02-16 16:29 
[DEBUG] finished: 5447/45794, elapsed: 0.07 hours, estimated: 02-16 16:33 
[DEBUG] finished: 5450/45794, elapsed: 0.07 hours, estimated: 02-16 16:33 
[DEBUG] finished: 5451/45794, elapsed: 0.07 hours, estimated: 02-16 16:33 
[DEBUG] finished: 5453/45794, elapsed: 0.07 hours, estimated: 02-16 16:33 
[DEBUG] finished: 5454/45794, elapsed: 0.07 hours, estimated: 02-16 16:33 
[DEBUG] finished: 5455/45794, elapsed: 0.07 hours, estimated: 02-16 16:34 
[DEBUG] finished: 5493/45

[INFO] workers finished 
[INFO] done post processing exp data 
=== Overall Report ===

project dir:	data/projs/verus_fast_unknown/base.z3
exper config:	verus_quick
solver path:	bin/z3-4.13.0
analyzer:	default

| category   |   count | percentage   |
|------------|---------|--------------|
| stable     |     186 | 73.52 %      |
| unstable   |      67 | 26.48 %      |
| total      |     253 | 100.00 %     |

=== Report End ===


CompletedProcess(args=['./src/exper_wizard.py', 'multiple', '-i', 'data/projs/verus_fast_unknown/base.z3/', '-e', 'verus_quick'], returncode=0)

In [5]:
#Runningn Mariposa on 
command = ["./src/exper_wizard.py", "multiple", "-i", "data/projs/verus_timeout/base.z3/", "-e", "verus_quick_less_muants"]
subprocess.run(command, check=True)



# will probably run this in a screen

[DEBUG] created table verus_timeout_base_z3_z3_4_13_0_exp 
[DEBUG] running ALL of verus_timeout.base.z3 
[DEBUG] adding 16340 tasks 


[INFO] running 860 original queries 
[INFO] running 16340 tasks 


[DEBUG] finished: 198/16341, elapsed: 0.03 hours, estimated: 02-16 19:07 
[DEBUG] finished: 199/16341, elapsed: 0.03 hours, estimated: 02-16 19:06 
[DEBUG] finished: 200/16341, elapsed: 0.03 hours, estimated: 02-16 19:05 
[DEBUG] finished: 201/16341, elapsed: 0.03 hours, estimated: 02-16 19:05 
[DEBUG] finished: 202/16341, elapsed: 0.03 hours, estimated: 02-16 19:04 
[DEBUG] finished: 204/16342, elapsed: 0.03 hours, estimated: 02-16 19:02 
[DEBUG] finished: 204/16341, elapsed: 0.03 hours, estimated: 02-16 19:02 


KeyboardInterrupt: 

In [7]:

command = ["./src/exper_wizard.py", "multiple", "-i", "data/projs/verus_slow_unknown/base.z3/", "-e", "verus_quick_less_muants", "--category-verbosity", "3"]
subprocess.run(command, check=True)

[DEBUG] created table verus_slow_unknown_base_z3_z3_4_13_0_exp 
[DEBUG] running ALL of verus_slow_unknown.base.z3 
[DEBUG] adding 19 tasks 


[INFO] running 1 original queries 
[INFO] running 19 tasks 


[DEBUG] worker 6 finished in 0.0 hours 
[DEBUG] worker 3 finished in 0.0 hours 
[DEBUG] worker 2 finished in 0.0 hours 
[DEBUG] worker 0 finished in 0.0 hours 
[DEBUG] worker 5 finished in 0.0 hours 
[DEBUG] worker 1 finished in 0.0 hours 
[DEBUG] worker 4 finished in 0.0 hours 


[INFO] workers finished 
[INFO] done post processing exp data 
=== Overall Report ===

project dir:	data/projs/verus_slow_unknown/base.z3
exper config:	verus_quick_less_muants
solver path:	bin/z3-4.13.0
analyzer:	default

| category   |   count | percentage   |
|------------|---------|--------------|
| stable     |       1 | 100.0 %      |
| total      |       1 | 100.00 %     |

=== Report End ===


CompletedProcess(args=['./src/exper_wizard.py', 'multiple', '-i', 'data/projs/verus_slow_unknown/base.z3/', '-e', 'verus_quick_less_muants', '--category-verbosity', '3'], returncode=0)

[DEBUG] finished: 632/16341, elapsed: 0.1 hours, estimated: 02-16 19:13 
